We do the CVE rate-outbreak for 2025 using logistic regression.

In [ ]:
# load environment
library(tidyverse)


In [ ]:
# loading data
outbreak <- read.csv("data/Outbreak.csv")
cve <- read.csv("data/CVE.csv")


In [ ]:
df <- cve %>%
  inner_join(outbreak, by = "County",
             suffix = c("_cve", "_out"))

In [ ]:
head(df)

,County,X2016_cve,X2017_cve,X2018_cve,X2019_cve,X2020_cve,X2021_cve,X2022_cve,X2023_cve,X2024_cve,...,X2016_out,X2017_out,X2018_out,X2019_out,X2020_out,X2021_out,X2022_out,X2023_out,X2024_out,X2025_out
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Anderson,0.35%,0.50%,0.75%,0.94%,1.00%,1.11%,1.62%,1.64%,2.05%,...,0,0,0,0,0,0,0,0,0,0
2,Andrews,0.75%,0.90%,1.07%,1.50%,0.39%,1.36%,0.45%,1.57%,1.43%,...,0,0,0,0,0,0,0,0,0,1
3,Angelina,0.70%,0.63%,0.67%,0.82%,0.96%,0.86%,1.03%,1.53%,1.98%,...,0,0,0,0,0,0,0,0,0,0
4,Aransas,1.22%,1.39%,1.49%,1.71%,1.57%,1.51%,0.00%,1.91%,1.91%,...,0,0,0,0,0,0,0,0,0,0
5,Archer,0.61%,0.45%,0.81%,1.43%,1.29%,1.56%,1.68%,2.32%,2.80%,...,0,0,0,0,0,0,0,0,0,0
6,Armstrong,0.59%,1.90%,1.44%,2.35%,3.53%,3.53%,3.77%,4.46%,4.95%,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df2025 <- df %>%
  transmute(
    County,
    cve = `X2025_cve`,
    outbreak = `X2025_out`
  )

In [ ]:
df2025$cve <- gsub("%", "", df2025$cve)
df2025$cve <- as.numeric(df2025$cve)

In [ ]:
# perform logistic regression
lr <- glm(outbreak ~ cve, data = df2025, family = "binomial")


In [ ]:
coef(lr)
summary(lr)

(Intercept)         cve 
 -2.2174213   0.1480485


Call:
glm(formula = outbreak ~ cve, family = "binomial", data = df2025)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -2.21742    0.31207  -7.105  1.2e-12 ***
cve          0.14805    0.07904   1.873    0.061 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 210.88  on 253  degrees of freedom
Residual deviance: 207.56  on 252  degrees of freedom
AIC: 211.56

Number of Fisher Scoring iterations: 4


In [ ]:
count <- read.csv("data/count.csv")
df <- cve %>%
  inner_join(count, by = "County",
             suffix = c("_cve", "_count"))
df2025 <- df %>%
  transmute(
    County,
    cve = `X2025_cve`,
    outbreak = `X2025_count`
  )
df2025$cve <- gsub("%", "", df2025$cve)
df2025$cve <- as.numeric(df2025$cve)
df2025$outbreak <- as.numeric(df2025$outbreak)
pr <- glm(outbreak ~ cve, data = df2025, family = "poisson")
coef(pr)
summary(pr)

(Intercept)         cve 
 -1.3823001   0.4866783


Call:
glm(formula = outbreak ~ cve, family = "poisson", data = df2025)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.382300   0.078298  -17.65   <2e-16 ***
cve          0.486678   0.007494   64.94   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 5469.3  on 253  degrees of freedom
Residual deviance: 2443.2  on 252  degrees of freedom
AIC: 2567.9

Number of Fisher Scoring iterations: 8


In [ ]:
enrollment <- read.csv("data/enrollment.csv")

enrollment <- enrollment %>%
  mutate(
    County = County %>%
      str_to_lower() %>%
      str_remove(" county") %>%
      str_trim() %>%
      str_to_title()
  )
enrollment <- enrollment[enrollment$County != "Grand Total", ]
View(enrollment)

,County,Enrollment.Sum
,<chr>,<int>
1,Anderson,7808
2,Andrews,4209
3,Angelina,15649
4,Aransas,2913
5,Archer,2110
6,Armstrong,297
7,Atascosa,9046
8,Austin,6290
9,Bailey,1330


In [ ]:
df2025 <- df2025 %>%
  left_join(enrollment, by = "County") %>%
  transmute(
    County,
    cve,
    outbreak,
    enrollment = Enrollment.Sum
  )
View(df2025)
pr2 <- glm(outbreak ~ cve, offset=log(df2025$enrollment), data = df2025, family = "poisson")
summary(pr2)

County,cve,outbreak,enrollment
<chr>,<dbl>,<dbl>,<int>
Anderson,2.54,0,7808
Andrews,1.91,3,4209
Angelina,2.50,0,15649
Aransas,2.06,0,2913
Archer,2.70,0,2110
Armstrong,5.24,0,297
Atascosa,1.08,1,9046
Austin,3.55,0,6290
Bailey,0.80,2,1330



Call:
glm(formula = outbreak ~ cve, family = "poisson", data = df2025, 
    offset = log(df2025$enrollment))

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -11.591582   0.072767 -159.30   <2e-16 ***
cve           0.638770   0.006728   94.94   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 7341.9  on 248  degrees of freedom
Residual deviance: 2525.2  on 247  degrees of freedom
  (5 Beobachtungen als fehlend gel"oscht)
AIC: 2645.8

Number of Fisher Scoring iterations: 8


In [ ]:
library(pscl)

In [ ]:
m1 <- glm(
  outbreak ~ cve + enrollment,
  family = poisson,
  data = df2025
)
summary(m1)


Call:
glm(formula = outbreak ~ cve + enrollment, family = poisson, 
    data = df2025)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.508e+00  8.273e-02 -18.232  < 2e-16 ***
cve          4.970e-01  7.672e-03  64.774  < 2e-16 ***
enrollment   2.992e-06  3.880e-07   7.711 1.25e-14 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 5437.8  on 248  degrees of freedom
Residual deviance: 2345.5  on 246  degrees of freedom
  (5 Beobachtungen als fehlend gel"oscht)
AIC: 2468.1

Number of Fisher Scoring iterations: 14


In [ ]:
df2025$enrollment <- as.numeric(df2025$enrollment)
stopifnot(all(is.finite(df2025$enrollment)), all(df2025$enrollment > 0))

pr2 <- glm(outbreak ~ cve + offset(log(enrollment)),
           family = poisson, data = df2025)

pR2(pr2)

ERROR: Error: all(is.finite(df2025$enrollment)) ist nicht TRUE


In [ ]:
m3 <- glm(
  outbreak ~ enrollment,
  family = poisson,
  data = df2025
)
summary(m3)


Call:
glm(formula = outbreak ~ enrollment, family = poisson, data = df2025)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.084e+00  3.841e-02  28.217   <2e-16 ***
enrollment  -1.302e-07  4.996e-07  -0.261    0.794    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 5437.8  on 248  degrees of freedom
Residual deviance: 5437.7  on 247  degrees of freedom
  (5 Beobachtungen als fehlend gel"oscht)
AIC: 5558.4

Number of Fisher Scoring iterations: 9


In [ ]:
library(MASS)

m_nb <- glm(
  outbreak ~ cve + log(enrollment),
  family = poisson,
  data = df2025
)
summary(m_nb)



Call:
glm(formula = outbreak ~ cve + log(enrollment), family = poisson, 
    data = df2025)

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -5.208155   0.268179  -19.42   <2e-16 ***
cve              0.516948   0.007784   66.41   <2e-16 ***
log(enrollment)  0.430505   0.025961   16.58   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 5437.8  on 248  degrees of freedom
Residual deviance: 2096.9  on 246  degrees of freedom
  (5 Beobachtungen als fehlend gel"oscht)
AIC: 2219.6

Number of Fisher Scoring iterations: 8


In [ ]:
m_nb_int <- glm.nb(
  outbreak ~ cve * log(enrollment),
  data = df2025
)
summary(m_nb_int)


Warning message:
"glm.fit: Algorithmus konvergierte nicht"



Call:
glm.nb(formula = outbreak ~ cve * log(enrollment), data = df2025, 
    init.theta = 0.05061484569, link = log)

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)   
(Intercept)         -7.93892    2.57585  -3.082  0.00206 **
cve                  1.65408    0.68334   2.421  0.01550 * 
log(enrollment)      0.82766    0.31186   2.654  0.00796 **
cve:log(enrollment) -0.15426    0.08766  -1.760  0.07846 . 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(0.0506) family taken to be 1)

    Null deviance: 108.508  on 248  degrees of freedom
Residual deviance:  82.258  on 245  degrees of freedom
  (5 Beobachtungen als fehlend gel"oscht)
AIC: 415.53

Number of Fisher Scoring iterations: 1


              Theta:  0.0506 
          Std. Err.:  0.0102 

 2 x log-likelihood:  -405.5290 